In [1]:
import polars as pl

In [2]:
health = pl.read_csv('./Data/health_survey.csv')
health.head()

Participant ID,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree"

In [3]:
reverse = pl.read_csv('./Data/ReverseCodingItems.csv')
reverse.head()

Question,Construct,Question # on Qualtrics Survey,Needs Reverse Coding?,Column Name
str,i64,i64,str,str
"""In the future, I plan to parti…",1,1,"""No""","""F1"""
"""Individuals are responsible fo…",5,2,"""Yes""","""F5"""
"""When tryng to understand the p…",2,3,"""No""","""F2"""
"""I plan to become involved in m…",1,4,"""No""","""F1.1"""
"""I can communicate well with ot…",2,5,"""No""","""F2.1"""


In [20]:
health_survey_summary = (health
 .select(['Participant ID', pl.col('^F.*$')])
 .unpivot(index = 'Participant ID',
          variable_name = 'Question',
          value_name = 'Response')
 .join(reverse.rename({'Column Name': 'Question', 'Question': 'Questions', 
                       'Needs Reverse Coding?' : 'Needs Reverse'}), on='Question', how='left')
 .select('Participant ID', 'Question', 'Response', 'Needs Reverse')
 .with_columns(
        pl.when(pl.col('Response') == 'Strongly Disagree').then(1)
        .when(pl.col('Response') == 'Somewhat Disagree').then(2)
        .when(pl.col('Response') == 'Neither Agree nor Disagree').then(3)
        .when(pl.col('Response') == 'Somewhat Agree').then(4)
        .when(pl.col('Response') == 'Strongly Agree').then(5)
        .alias('Temp Coded Value'))
  .with_columns(
        (6 - pl.col('Temp Coded Value')).alias('Temp Reversed Coded Value'))
    .with_columns(
        pl.when(pl.col('Needs Reverse') == 'Yes').then(pl.col('Temp Reversed Coded Value'))
        .otherwise(pl.col('Temp Coded Value')).alias('Recoded Value'))
        .with_columns(pl.col('Question').str.extract(r'(\w+)', 1).alias('Question Type'))
        .select('Participant ID', 'Question Type', 'Recoded Value') 
        .group_by(pl.col('Participant ID'), pl.col('Question Type'))
        .agg(pl.col('Recoded Value').sum().alias('Recoded Total'))
        .pivot('Question Type', 
                index = 'Participant ID', 
                values = 'Recoded Total')
    )  
health_survey_summary.head()

Participant ID,F2,F3,F6,F1,F5,F4
i64,i32,i32,i32,i32,i32,i32
24,51,17,21,34,28,20
98,58,18,17,36,37,24
198,47,21,19,35,29,14
10,58,28,23,35,40,19
127,49,20,20,31,32,17


In [21]:
health_survey_summary.write_csv('./data/health_survey_summary.csv')